# Movie Recommendation System

## 1. Import libraries

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ast 
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise.model_selection import KFold

import warnings; warnings.simplefilter('ignore')

## 2. Load dataset

We have MovieLens datasets.

**The Full Dataset:** Consists of 26,000,000 ratings and 750,000 tag applications applied to 45,000 movies by 270,000 users. Includes tag genome data with 12 million relevance scores across 1,100 tags.

**The Small Dataset:** Comprises of 100,000 ratings and 1,300 tag applications applied to 9,000 movies by 700 users.

We will build our Simple Recommender using movies from the Full Dataset 

In [2]:
credits = pd.read_csv('D:/input_data/movie_dataset/credits.csv')
keywords = pd.read_csv('D:/input_data/movie_dataset/keywords.csv')
links_small = pd.read_csv('D:/input_data/movie_dataset/links_small.csv')
md = pd.read_csv('D:/input_data/movie_dataset/movies_metadata.csv')
ratings = pd.read_csv('D:/input_data/movie_dataset/ratings_small.csv')
links =  pd.read_csv('D:/input_data/movie_dataset/links.csv')

## 3. Explore Dataset

#### Credits dataframe

In [3]:
credits.head()
#credits.iloc[0:3]
#credits['cast'].iloc[0:3]
#credits.iloc[:,0:2]

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [4]:
credits.columns

Index(['cast', 'crew', 'id'], dtype='object')

* **cast:** Information about casting. Name of actor, gender and it's character name in movie
* **crew:** Information about crew members. Like who directed the movie, editor of the movie and so on. 
* **id:** It's movie ID given by TMDb

In [5]:
credits.shape

(45476, 3)

In [6]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


#### Keywords dataframe

In [7]:
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [8]:
keywords.columns

Index(['id', 'keywords'], dtype='object')

* **id:** It's movie ID given by TMDb
* **Keywords:** Tags/keywords for the movie. It list of tags/keywords 

In [9]:
keywords.shape

(46419, 2)

In [10]:
keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


#### Link dataframe

In [11]:
links_small.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [12]:
links_small.columns

Index(['movieId', 'imdbId', 'tmdbId'], dtype='object')

* **movieId:** It's serial number for movie
* **imdbId:** Movie id given on IMDb platform
* **tmdbId**: Movie id given on TMDb platform

In [13]:
links_small.shape

(9125, 3)

In [14]:
links_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9125 non-null   int64  
 1   imdbId   9125 non-null   int64  
 2   tmdbId   9112 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 214.0 KB


In [15]:
md.iloc[0:3].transpose()

,0,1,2
adult,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...",NaN,"{'id': 119050, 'name': 'Grumpy Old Men Collect..."
budget,30000000,65000000,0
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
homepage,http://toystory.disney.com/toy-story,NaN,NaN
id,862,8844,15602
imdb_id,tt0114709,tt0113497,tt0113228
original_language,en,en,en
original_title,Toy Story,Jumanji,Grumpier Old Men
overview,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...,A family wedding reignites the ancient feud be...


In [16]:
md.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

**Features**

* **adult:** Indicates if the movie is X-Rated or Adult.
* **belongs_to_collection:** A stringified dictionary that gives information on the movie series the particular film belongs to.
* **budget:** The budget of the movie in dollars.
* **genres:** A stringified list of dictionaries that list out all the genres associated with the movie.
* **homepage:** The Official Homepage of the move.
* **id:** The ID of the movie.
* **imdb_id:** The IMDB ID of the movie.
* **original_language:** The language in which the movie was originally shot in.
* **original_title:** The original title of the movie.
* **overview:** A brief blurb of the movie.
* **popularity:** The Popularity Score assigned by TMDB.
* **poster_path:** The URL of the poster image.
* **production_companies:** A stringified list of production companies involved with the making of the movie.
* **production_countries:** A stringified list of countries where the movie was shot/produced in.
* **release_date:** Theatrical Release Date of the movie.
* **revenue:** The total revenue of the movie in dollars.
* **runtime:** The runtime of the movie in minutes.
* **spoken_languages:** A stringified list of spoken languages in the film.
* **status:** The status of the movie (Released, To Be Released, Announced, etc.)
* **tagline:** The tagline of the movie.
* **title:** The Official Title of the movie.
* **video:** Indicates if there is a video present of the movie with TMDB.
* **vote_average:** The average rating of the movie.
* **vote_count:** The number of votes by users, as counted by TMDB.

In [17]:
md.shape

(45466, 24)

In [18]:
md.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

#### Ratings dataframe

In [19]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [20]:
ratings.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

* **userId:** It is id for User
* **movieId:** It is TMDb movie id.
* **rating:** Rating given for the particular movie by specific user
* **timestamp:** Time stamp when rating has been given by user

In [21]:
ratings.shape

(100004, 4)

In [22]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


## 4. Pre-processing

#### We will process data during training when necessary

## 5. Build Movie Recommendation System

### 5.1. Simple recommendation system

**Approach: **

* The Simple Recommender offers __generalized recommendations__ to every user __based on movie popularity and (sometimes) genre__. 

* The __basic idea__ behind this recommender is that __movies that are more popular and more critically acclaimed will have a higher probability of being liked by the average audience.__ 

* This model __does not give personalized recommendations__ based on the user.

**What we are actually doing: **

* The implementation of this model is extremely trivial. 
* All we have to do is __sort our movies based on ratings and popularity__ and display the top movies of our list. 
* As an added step, we can __pass in a genre argument to get the top movies of a particular genre.__

In [23]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i[
    'name'] for i in x] if isinstance(x, list) else [])

* I use the TMDB Ratings to come up with our Top Movies Chart. 
* I will use IMDB's weighted rating formula to construct my chart.
* Mathematically, it is represented as follows:



$\large Weighted\; Rating (WR) = (\frac{v}{v + m} . R) + (\frac{m}{v + m} . C)$
```
where,
    v is the number of votes for the movie
    m is the minimum votes required to be listed in the chart
    R is the average rating of the movie
    C is the mean vote across the whole report
```

In [24]:
#V:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
#R:
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
#C:
C = vote_averages.mean()
C

5.244896612406511

* The next step, we need to determine an appropriate value for `m`, the minimum votes required to be listed in the chart. 

* We will use 95th percentile as our cutoff. In other words, for a movie to feature in the charts, it must have more votes than at least 95% of the movies in the list.

In [25]:
m = vote_counts.quantile(0.95)
m

434.0

In [26]:
# Pre-processing step for getting year from date by splliting it using '-'
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(
    lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [27]:
qualified = md[(md['vote_count'] >= m) & 
               (md['vote_count'].notnull()) & 
               (md['vote_average'].notnull())][['title', 
                                                'year', 
                                                'vote_count', 
                                                'vote_average', 
                                                'popularity', 
                                                'genres']]

qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 6)

* Therefore, to qualify to be considered for the chart, a movie has to have at least __434 votes__ on TMDB. 
* We also see that the __average rating__ for __a movie on TMDB__ is __5.244 on a scale of 10__. 
* Here, only __2274 movies__ are qualify to be on our chart.

In [28]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [29]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [30]:
qualified = qualified.sort_values('wr', ascending=False).head(250)

**Top Movies**

In [31]:
qualified.head(15)

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.108149,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167259,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.213481,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.869599,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.950236,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.645403,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.307194,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,"[Adventure, Fantasy, Action]",7.851924


* We see that three Christopher Nolan Films, __Inception__, __The Dark Knight__ and __Interstellar__ occur at the very top of our chart. 
* The chart also indicates a strong bias of TMDB Users towards particular genres and directors.

* Let us now construct our __function that builds charts for particular genres.__

* For this, we __relax__ our __default conditions to the 85th percentile instead of 95.__

In [32]:
'''
>>> s
     a   b
one  1.  2.
two  3.  4.

>>> s.stack()
one a    1
    b    2
two a    3
    b    4
'''
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)
gen_md.head(3).transpose()

,0,0,0
adult,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...","{'id': 10194, 'name': 'Toy Story Collection', ...","{'id': 10194, 'name': 'Toy Story Collection', ..."
budget,30000000,30000000,30000000
homepage,http://toystory.disney.com/toy-story,http://toystory.disney.com/toy-story,http://toystory.disney.com/toy-story
id,862,862,862
imdb_id,tt0114709,tt0114709,tt0114709
original_language,en,en,en
original_title,Toy Story,Toy Story,Toy Story
overview,"Led by Woody, Andy's toys live happily in his ...","Led by Woody, Andy's toys live happily in his ...","Led by Woody, Andy's toys live happily in his ..."
popularity,21.946943,21.946943,21.946943


In [33]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & 
                   (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: 
                        (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C),
                        axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

Let us see our method in action by displaying the __Top 15 Romance Movies__ (Romance almost didn't feature at all in our Generic Top Chart despite being one of the most popular movie genres).


**Top 15 Romantic Movies**

In [34]:
build_chart('Romance').head(15)

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457024,8.565285
351,Forrest Gump,1994,8147,8,48.307194,7.971357
876,Vertigo,1958,1162,8,18.20822,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.845107,7.745154
1132,Cinema Paradiso,1988,834,8,14.177005,7.744878
19901,Paperman,2012,734,8,7.198633,7.713951
37863,Sing Street,2016,669,8,10.672862,7.689483
882,The Apartment,1960,498,8,11.994281,7.599317
38718,The Handmaiden,2016,453,8,16.727405,7.566166


### 5.2 Content based recommendation system

In [35]:
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [36]:
## Pre-processing step

def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan


In [37]:
md['id'] = md['id'].apply(convert_int)
md[md['id'].isnull()]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[Carousel Productions, Vision View Entertainme...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",NaN,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[Aniplex, GoHands, BROSTA TV, Mardock Scramble...","[{'iso_3166_1': 'US', 'name': 'United States o...",NaN,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[Odyssey Media, Pulser Productions, Rogue Stat...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",NaN,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [38]:
md = md.drop([19730, 29503, 35587])

In [39]:
md['id'] = md['id'].astype('int')

In [40]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 25)

We have __9099 movies__ available in our small movies metadata dataset which is 5 times smaller than our original dataset of 45000 movies.

###  Content based recommendation system : Using movie description and taglines

* Let us first try to build a recommender using movie descriptions and taglines. 
* We do not have a quantitative metric to judge our machine's performance so this will have to be done 
  qualitatively.

In [41]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [42]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0.0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [43]:
tfidf_matrix.shape

(9099, 268124)

* Since we have used the TF-IDF Vectorizer, calculating the Dot Product will directly give us the Cosine Similarity Score. 

* Therefore, we will use sklearn's linear_kernel instead of cosine_similarities since it is much faster.

In [44]:
# http://scikit-learn.org/stable/modules/metrics.html#linear-kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [45]:
cosine_sim[0]
#cosine_sim.shape

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

* We now have a pairwise cosine similarity matrix for all the movies in our dataset. 
* The next step is to write a function that returns the 30 most similar movies based on the cosine similarity score.

In [46]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])
#indices.head(2)

In [47]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

* We're all set...!
* Let us now try and get the top recommendations for a few movies and see how good the recommendations are.

In [48]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [49]:
get_recommendations('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

* We see that for The __Dark Knight__, our system is able to identify it as a __Batman film and subsequently recommend other Batman films__ as its top recommendations.

* But unfortunately, that is all this system can do at the moment. 

* This is not of much use to most people as it doesn't take into considerations very important features such as cast, crew, director and genre, which determine the rating and the popularity of a movie. 

* Someone who liked The Dark Knight probably likes it more because of Nolan and would hate Batman Forever and every other substandard movie in the Batman Franchise.

* Therefore, we are going to use much more suggestive metadata than Overview and Tagline. 
* In the next subsection, we will build a more sophisticated recommender that takes __genre, keywords, cast and crew__ into consideration.

### Content based RS : Using movie description, taglines, keywords, cast, director and genres

In [51]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [52]:
md.shape

(45463, 25)

In [53]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [54]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 28)

We now have our cast, crew, genres and credits, all in one dataframe. Let us wrangle this a little more using the following intuitions:

**1. Crew:** From the crew, we will only pick the director as our feature since the others don't contribute that much to the feel of the movie.

**2. Cast:** Choosing Cast is a little more tricky. Lesser known actors and minor roles do not really affect people's opinion of a movie. Therefore, we must only select the major characters and their respective actors. Arbitrarily we will choose the top 3 actors that appear in the credits list.

In [55]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))


In [56]:
smd['cast']

0        [{'cast_id': 14, 'character': 'Woody (voice)',...
1        [{'cast_id': 1, 'character': 'Alan Parrish', '...
2        [{'cast_id': 2, 'character': 'Max Goldman', 'c...
3        [{'cast_id': 1, 'character': 'Savannah 'Vannah...
4        [{'cast_id': 1, 'character': 'George Banks', '...
                               ...                        
40952    [{'cast_id': 1, 'character': 'Henry Cobb', 'cr...
41172    [{'cast_id': 0, 'character': 'Rustom Pavri', '...
41225    [{'cast_id': 0, 'character': 'Sarman', 'credit...
41391    [{'cast_id': 4, 'character': 'Rando Yaguchi : ...
41669    [{'cast_id': 0, 'character': 'Himself', 'credi...
Name: cast, Length: 9219, dtype: object

In [57]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [58]:
smd['director'] = smd['crew'].apply(get_director)
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

* Approach to building the recommender is going to be extremely hacky. 

* What I plan on doing is creating a metadata dump for every movie which consists of genres, director, main actors and keywords. 

* I then use a __Count Vectorizer__ to create our __count matrix__

* The remaining steps are similar to what we did earlier: we calculate the cosine similarities and return movies that are most similar.

These are steps I follow in the preparation of my genres and credits data:

1. __Strip Spaces and Convert to Lowercase__ from all our features. This way, our engine will not confuse between __Johnny Depp and Johnny Galecki.__
2. __Mention Director 2 times__ to give it __more weight relative to the entire cast.__

In [59]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

**Keywords**

* We will do a small amount of pre-processing of our keywords before putting them to any use. 
* we __calculate the frequenct counts of every keyword__ that appears in the dataset.

In [60]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s = s.value_counts()
s[:5]

keyword
independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: count, dtype: int64

* Keywords occur in frequencies ranging from 1 to 610.
* We do not have any use for keywords that occur only once. 
* Therefore, these can be safely removed. 
* Finally, we will convert every word to its stem so that words such as __Dogs__ and __Dog__ are considered the same.

In [61]:
s = s[s > 1]

In [62]:
# Just an example
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [63]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [64]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [65]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [66]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0.0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [67]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [68]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

* We will reuse the get_recommendations function that we had written earlier. 
* Since our cosine similarity scores have changed, we expect it to give us different (and probably better) results.
* Let us check for __The Dark Knight__ again and see what recommendations I get this time around.

In [69]:
get_recommendations('The Dark Knight').head(10)

8031         The Dark Knight Rises
6218                 Batman Begins
6623                  The Prestige
2085                     Following
7648                     Inception
4145                      Insomnia
3381                       Memento
8613                  Interstellar
7659    Batman: Under the Red Hood
1134                Batman Returns
Name: title, dtype: object

* I am much more satisfied with the results I get this time around. The recommendations seem to have recognized other Christopher Nolan movies (due to the high weightage given to director) and put them as top recommendations. 
* I enjoyed watching __The Dark Knight__ as well as some of the other ones in the list including __Batman Begins, The Prestige and The Dark Knight Rises.__



**Improvment** 

* We can of course experiment on this engine by trying out different weights for our features (directors, actors, genres), limiting the number of keywords that can be used in the soup, weighing genres based on their frequency, only showing movies with the same languages, etc.

In [70]:
get_recommendations('Inception').head(10)

6623                             The Prestige
3381                                  Memento
4145                                 Insomnia
2085                                Following
8031                    The Dark Knight Rises
8613                             Interstellar
6981                          The Dark Knight
6218                            Batman Begins
5638    Sky Captain and the World of Tomorrow
8500                                  Don Jon
Name: title, dtype: object

In [71]:
get_recommendations('Mean Girls').head(10)

3319               Head Over Heels
4763                 Freaky Friday
1329              The House of Yes
6277              Just Like Heaven
7905         Mr. Popper's Penguins
7332    Ghosts of Girlfriends Past
6959     The Spiderwick Chronicles
8883                      The DUFF
6698         It's a Boy Girl Thing
7377       I Love You, Beth Cooper
Name: title, dtype: object

In [72]:
get_recommendations('Pulp Fiction').head(10)

1381            Jackie Brown
8905       The Hateful Eight
5200       Kill Bill: Vol. 2
898           Reservoir Dogs
4903       Kill Bill: Vol. 1
7280    Inglourious Basterds
6788             Death Proof
8310        Django Unchained
4595                   Basic
4764                S.W.A.T.
Name: title, dtype: object

#### Add Popularity and Ratings 

* One thing that we notice about our recommendation system is that it recommends movies regardless of ratings and 
  popularity. It is true that Batman and Robin has a lot of similar characters as compared to The Dark Knight but   
  it was a terrible movie that shouldn't be recommended to anyone.

* Therefore, we will add a mechanism to remove bad movies and return movies which are popular and have had a good
  critical response.

* I will take the top 25 movies based on similarity scores and calculate the vote of the __60th percentile__ movie. 
  Then, using this as the value of $m$, we will calculate the weighted rating of each movie using IMDB's formula 
  like we did in the Simple Recommender section.

In [73]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & 
                       (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [74]:
improved_recommendations('The Dark Knight')

,title,vote_count,vote_average,year,wr
7648,Inception,14075,8,2010,7.917588
8613,Interstellar,11187,8,2014,7.897107
6623,The Prestige,4510,8,2006,7.758148
3381,Memento,4168,8,2000,7.740175
8031,The Dark Knight Rises,9263,7,2012,6.921448
6218,Batman Begins,7511,7,2005,6.904127
1134,Batman Returns,1706,6,1992,5.846862
132,Batman Forever,1529,5,1995,5.054144
9024,Batman v Superman: Dawn of Justice,7189,5,2016,5.013943
1260,Batman & Robin,1447,4,1997,4.287233


In [75]:
improved_recommendations('Pulp Fiction')

,title,vote_count,vote_average,year,wr
898,Reservoir Dogs,3821,8,1992,7.718986
8310,Django Unchained,10297,7,2012,6.929017
7280,Inglourious Basterds,6598,7,2009,6.891679
4903,Kill Bill: Vol. 1,5091,7,2003,6.862133
8905,The Hateful Eight,4405,7,2015,6.842588
5200,Kill Bill: Vol. 2,4061,7,2004,6.830542
1381,Jackie Brown,1580,7,1997,6.621790
65,From Dusk Till Dawn,1644,6,1996,5.842293
6788,Death Proof,1359,6,2007,5.817225
4764,S.W.A.T.,780,5,2003,5.087550


* Unfortunately, __Batman and Robin__ does not disappear from our recommendation list. 
* This is probably due to the fact that it is rated a 4, which is only slightly below average on TMDB. 
* It certainly doesn't deserve a 4 when amazing movies like The Dark Knight Rises has only a 7. 
* However, there is nothing much we can do about this. Therefore, we will conclude our Content Based Recommender 
  section here 

### 5.3 CF based recommendation system

**Our content based engine suffers from some severe limitations.**

* It is only capable of suggesting movies which are close to a certain movie. That is, it is not capable of capturing tastes and providing recommendations across genres.



* Also, the engine that we built is not really personal in that it doesn't capture the personal tastes and biases of a user. Anyone querying our engine for recommendations based on a movie will receive the same recommendations for that movie, regardless of who (s)he is.



* Therefore, in this section, we will use Collaborative Filtering to make recommendations to Movie Watchers. Collaborative Filtering is based on the idea that users similar to a me can be used to predict how much I will like a particular product or service those users have used/experienced but I have not.



* I will not be implementing Collaborative Filtering from scratch. Instead, I will use the __[Surprise library](http://surpriselib.com/)__ that used extremely powerful algorithms like __Singular Value Decomposition (SVD) to minimise RMSE (Root Mean Square Error) and give great recommendations__.



* Implementation of SVD for surprise library is given on this [link](https://github.com/NicolasHug/Surprise/blob/master/surprise/prediction_algorithms/matrix_factorization.pyx)

In [77]:
# surprise reader API to read the dataset
reader = Reader()

In [78]:
from surprise.model_selection import KFold

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
kf = KFold(n_splits=5)

In [79]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import cross_validate
# Define the SVD algorithm
svd = SVD()

# Perform cross-validation and evaluate the algorithm
results = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Access the evaluation results
print('RMSE:', results['test_rmse'].mean())
print('MAE:', results['test_mae'].mean())


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8929  0.8965  0.8985  0.8987  0.8952  0.8964  0.0022  
MAE (testset)     0.6877  0.6931  0.6901  0.6922  0.6904  0.6907  0.0019  
Fit time          1.73    1.93    1.95    2.12    2.16    1.98    0.15    
Test time         0.33    0.27    0.27    0.26    0.28    0.28    0.02    
RMSE: 0.896352864368146
MAE: 0.6906864710321907


In [80]:
trainset = data.build_full_trainset()
#svd.train(trainset)

In [81]:
for trainset, testset in kf.split(data):
    # Fit the algorithm on the trainset
    svd.fit(trainset)
    
    # Make predictions on the testset
    predictions = svd.test(testset)
    
    # Compute and print the evaluation metrics
    accuracy.rmse(predictions)
    accuracy.mae(predictions)

RMSE: 0.8955
MAE:  0.6880
RMSE: 0.8984
MAE:  0.6938
RMSE: 0.8985
MAE:  0.6925
RMSE: 0.8967
MAE:  0.6877
RMSE: 0.8932
MAE:  0.6895


In [82]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [83]:
svd.predict(1, 302)

Prediction(uid=1, iid=302, r_ui=None, est=2.959189532159773, details={'was_impossible': False})

* For movie with ID 302, we get an estimated prediction of 2.686. One startling feature of this recommender system is that it doesn't care what the movie is (or what it contains). It works purely on the basis of an assigned movie ID and tries to predict ratings based on how the other users have perceive the movie.

### Hybrid recommendation system

* In this section, will try to build a simple hybrid recommender that brings together techniques we have implemented in the content based and collaborative filter based engines. This is how it will work:

* **Input:** User ID and the Title of a Movie
* **Output:** Similar movies sorted on the basis of expected ratings by that particular user.

In [84]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [85]:
id_map = pd.read_csv('D:/input_data/movie_dataset/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')
#id_map = id_map.set_index('tmdbId')

In [86]:
indices_map = id_map.set_index('id')

In [87]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    movie_id = id_map.loc[title]['movieId']
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'release_date', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [88]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,release_date,id,est
522,Terminator 2: Judgment Day,4274.0,7.7,1991-07-01,280,3.354941
1011,The Terminator,4208.0,7.4,1984-10-26,218,3.352924
974,Aliens,3282.0,7.7,1986-07-18,679,3.255080
922,The Abyss,822.0,7.1,1989-08-09,2756,3.195709
8401,Star Trek Into Darkness,4479.0,7.4,2013-05-05,54138,3.157457
1621,Darby O'Gill and the Little People,35.0,6.7,1959-06-29,18887,3.122807
8658,X-Men: Days of Future Past,6155.0,7.5,2014-05-15,127585,3.089384
344,True Lies,1138.0,6.8,1994-07-14,36955,2.973878
4017,Hawk the Slayer,13.0,4.5,1980-08-27,25628,2.945278
6084,Beastmaster 2: Through the Portal of Time,17.0,4.6,1991-06-08,27549,2.914217


## Matrix Factorization Collaborative Filtering

In [89]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 

class MF(object):
    def __init__(self, Y_data, K, lam = 0.1, Xinit = None, Winit = None, 
                 learning_rate = 0.5, max_iter = 1000, print_every = 100, user_based = 1):
        self.Y_raw_data = Y_data
        self.K = K
        self.lam = lam
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.print_every = print_every
        self.user_based = user_based
        # number of users and items. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(Y_data[:, 0])) + 1 
        self.n_items = int(np.max(Y_data[:, 1])) + 1
        
        if Xinit is None: 
            self.X = np.random.randn(self.n_items, K)
        else:
            self.X = Xinit 
        
        if Winit is None: 
            self.W = np.random.randn(K, self.n_users)
        else: 
            self.W = Winit
            
        #self.all_users = self.Y_data[:,0] # all users (may be duplicated)
        self.n_ratings = Y_data.shape[0]
        # normalized data
        self.Y_data_n = self.Y_raw_data.copy()

    def normalize_Y(self):
        if self.user_based:
            user_col = 0
            item_col = 1
            n_objects = self.n_users
        else:
            user_col = 1
            item_col = 0 
            n_objects = self.n_items

        users = self.Y_raw_data[:, user_col] 
        self.mu = np.zeros((n_objects,))
        for n in range(n_objects):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data_n[ids, item_col] 
            # and the corresponding ratings 
            ratings = self.Y_data_n[ids, 2]
            # take mean
            m = np.mean(ratings) 
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            self.mu[n] = m
            # normalize
            self.Y_data_n[ids, 2] = ratings - self.mu[n]
            
    
    def loss(self):
        L = 0 
        for i in range(self.Y_data_n.shape[0]):
            # user, item, rating
            n, m, rate = int(self.Y_data_n[i, 0]), int(self.Y_data_n[i, 1]), self.Y_data_n[i, 2]
            L += 0.5*(rate - self.X[m, :].dot(self.W[:, n]))**2
            
        # regularization, don't ever forget this 
        L /= self.n_ratings
        L += 0.5*self.lam*(np.linalg.norm(self.X, 'fro') + np.linalg.norm(self.W, 'fro'))
        return L 

    
    def get_items_rated_by_user(self, user_id):
        """
        get all items which are rated by user n, and the corresponding ratings
        """
        # y = self.Y_data_n[:,0] # all users (may be duplicated)
        # item indices rated by user_id
        # we need to +1 to user_id since in the rate_matrix, id starts from 1 
        # while index in python starts from 0
        ids = np.where(self.Y_data_n[:,0] == user_id)[0] 
        item_ids = self.Y_data_n[ids, 1].astype(np.int32) # index starts from 0 
        ratings = self.Y_data_n[ids, 2]
        return (item_ids, ratings)
        
        
    def get_users_who_rate_item(self, item_id):
        """
        get all users who rated item m and get the corresponding ratings
        """
        ids = np.where(self.Y_data_n[:,1] == item_id)[0] 
        user_ids = self.Y_data_n[ids, 0].astype(np.int32)
        ratings = self.Y_data_n[ids, 2]
        return (user_ids, ratings)
        
    def updateX(self):
        for m in range(self.n_items):
            user_ids, ratings = self.get_users_who_rate_item(m)
            Wm = self.W[:, user_ids]
            grad_xm = -(ratings - self.X[m, :].dot(Wm)).dot(Wm.T)/self.n_ratings + \
                                               self.lam*self.X[m, :]
            self.X[m, :] -= self.learning_rate*grad_xm.reshape((self.K,))
    
    def updateW(self):
        for n in range(self.n_users):
            item_ids, ratings = self.get_items_rated_by_user(n)
            Xn = self.X[item_ids, :]
            grad_wn = -Xn.T.dot(ratings - Xn.dot(self.W[:, n]))/self.n_ratings + \
                        self.lam*self.W[:, n]
            self.W[:, n] -= self.learning_rate*grad_wn.reshape((self.K,))
    
    def fit(self):
        self.normalize_Y()
        for it in range(self.max_iter):
            self.updateX()
            self.updateW()
            if (it + 1) % self.print_every == 0:
                rmse_train = self.evaluate_RMSE(self.Y_raw_data)
                print ('iter =', it + 1, ', loss =', self.loss(), ', RMSE train =', rmse_train)
    
    
    def pred(self, u, i):
        """ 
        predict the rating of user u for item i 
        if you need the un
        """
        u = int(u)
        i = int(i)
        
        if self.user_based:
            bias = self.mu[u]
        else: 
            bias = self.mu[i]
        pred = self.X[i, :].dot(self.W[:, u]) + bias 
        if pred < 1:
            return 1 
        if pred > 5: 
            return 5 
        return pred 
        
    
    def pred_for_user(self, user_id):
        ids = np.where(self.Y_data_n[:, 0] == user_id)[0]
        items_rated_by_u = self.Y_data_n[ids, 1].tolist()              
        
        y_pred = self.X.dot(self.W[:, user_id]) + self.mu[user_id]
        predicted_ratings= []
        for i in range(self.n_items):
            if i not in items_rated_by_u:
                predicted_ratings.append((i, y_pred[i]))
        return predicted_ratings
    
    def evaluate_RMSE(self, rate_test):
        n_tests = rate_test.shape[0]
        SE = 0 # squared error
        for n in range(n_tests):
            pred = self.pred(rate_test[n, 0], rate_test[n, 1])
            SE += (pred - rate_test[n, 2])**2 

        RMSE = np.sqrt(SE/n_tests)
        return RMSE


In [91]:
import pandas as pd
import numpy as np

# Define the column names
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

# Read the data from the CSV file and set the correct delimiter (comma ',')
#ratings_base = pd.read_csv('D:/input_data/movie_dataset/ratings_small.csv', sep=',', names=r_cols, encoding='latin-1', header=None, skiprows=1)
ratings_base = pd.read_csv('D:/input_data/movie_dataset/ratings_small.csv')
# Convert the 'user_id' and 'movie_id' columns to integers
ratings_base['userId'] = ratings_base['userId'].astype(int)
ratings_base['movieId'] = ratings_base['movieId'].astype(int)

# Subtract 1 from the 'user_id' and 'movie_id' columns
ratings_base[['userId', 'movieId']] -= 1

# Convert the DataFrame to a NumPy array
ratings = ratings_base.values

ratings

array([[0.00000000e+00, 3.00000000e+01, 2.50000000e+00, 1.26075914e+09],
       [0.00000000e+00, 1.02800000e+03, 3.00000000e+00, 1.26075918e+09],
       [0.00000000e+00, 1.06000000e+03, 3.00000000e+00, 1.26075918e+09],
       ...,
       [6.70000000e+02, 6.36400000e+03, 4.00000000e+00, 1.07094036e+09],
       [6.70000000e+02, 6.38400000e+03, 2.50000000e+00, 1.07097966e+09],
       [6.70000000e+02, 6.56400000e+03, 3.50000000e+00, 1.07478472e+09]])

In [92]:
from sklearn.model_selection import train_test_split

# Assuming you have already defined 'ratings', which is the NumPy array of the dataset
rate_train, rate_test = train_test_split(ratings, test_size=0.33, random_state=42)
print(rate_train.shape, rate_test.shape)


(67002, 4) (33002, 4)


In [94]:
rs = MF(rate_train, K = 2, lam = 0.1, print_every = 2, learning_rate = 2, max_iter = 20, user_based = 0)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print ('\nItem-based MF, RMSE =', RMSE)

iter = 2 , loss = 19.9941134373712 , RMSE train = 1.0262607765575995
iter = 4 , loss = 12.863006045546694 , RMSE train = 0.9109145160148525
iter = 6 , loss = 8.360095718360997 , RMSE train = 0.8875413763569248
iter = 8 , loss = 5.488454727966506 , RMSE train = 0.8834268505617452
iter = 10 , loss = 3.6523562277599524 , RMSE train = 0.882690784387632
iter = 12 , loss = 2.47756247064308 , RMSE train = 0.882550166879924
iter = 14 , loss = 1.7257520428481223 , RMSE train = 0.882519837446111
iter = 16 , loss = 1.2446051704757963 , RMSE train = 0.8825121348453433
iter = 18 , loss = 0.9366739951123895 , RMSE train = 0.8825098319562713
iter = 20 , loss = 0.7395988657022685 , RMSE train = 0.8825090563361842

Item-based MF, RMSE = 1.0961680086368122


In [95]:
RMSE = rs.evaluate_RMSE(rate_train)
print (RMSE)

0.8825090563361842


In [96]:
data = rs.pred_for_user(1)
# Sort the data based on the absolute difference between the rating and 5.0 (in ascending order)
sorted_data = sorted(data, key=lambda x: abs(x[1] - 5.0))

# Get the top 20 IDs and ratings closest to 5.0
top_20_ids_ratings = sorted_data[:20]

# Print the result
for item in top_20_ids_ratings:
    print(f"ID: {item[0]}, Rating: {item[1]}")

ID: 21192, Rating: 3.4432708893371102
ID: 104109, Rating: 3.4432435914335886
ID: 125907, Rating: 3.443239780018727
ID: 76510, Rating: 3.443228959622556
ID: 99273, Rating: 3.443216687555512
ID: 13819, Rating: 3.443187782835972
ID: 160957, Rating: 3.4431745720877442
ID: 145703, Rating: 3.4431684541075267
ID: 89693, Rating: 3.443166568632548
ID: 149385, Rating: 3.443163469044272
ID: 88712, Rating: 3.4431606248253357
ID: 22410, Rating: 3.4431535454259348
ID: 77223, Rating: 3.443152584579546
ID: 63350, Rating: 3.443152486955914
ID: 42797, Rating: 3.4431448954173938
ID: 59086, Rating: 3.4431358102997858
ID: 92397, Rating: 3.4431341499850014
ID: 80682, Rating: 3.443134116266708
ID: 141250, Rating: 3.4431335142947996
ID: 64149, Rating: 3.4431322892282807


In [99]:
def hybridMatrix(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    movie_id = id_map.loc[title]['movieId']
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'release_date', 'id']]
    movies['est'] = movies['id'].apply(lambda x: rs.pred(userId, indices_map.loc[x]['movieId']))
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [100]:
hybridMatrix(1, 'Avatar')

,title,vote_count,vote_average,release_date,id,est
922,The Abyss,822.0,7.1,1989-08-09,2756,3.999891
974,Aliens,3282.0,7.7,1986-07-18,679,3.851303
4347,Piranha Part Two: The Spawning,41.0,3.9,1981-01-01,31646,3.833557
1500,Small Soldiers,522.0,6.2,1998-07-10,11551,3.772762
522,Terminator 2: Judgment Day,4274.0,7.7,1991-07-01,280,3.740327
1376,Titanic,7770.0,7.5,1997-11-18,597,3.359163
2014,Fantastic Planet,140.0,7.6,1973-05-01,16306,3.356881
1011,The Terminator,4208.0,7.4,1984-10-26,218,3.285723
831,Escape to Witch Mountain,60.0,6.5,1975-03-21,14821,3.099862
344,True Lies,1138.0,6.8,1994-07-14,36955,3.023824
